# 利用两端时间$t_u$，$t_d$确定粒子入射位置$t_x$

## 打开root文件，继承tree中Branch。

In [1]:
%jsroot on
TCanvas *c1=new TCanvas("c1","c1");//创建画布
TH1D *tdiff=new TH1D("tdiff","td-tu",140,-20,50);  //创建时间差histogram

//声明branch变量
Double_t tu, td, x;
//Double_t ctof,tof,x;
//int pid;

//打开root文件
TFile *ipf=new TFile("./tree.root");//打开ROOT文件
//得到tree的指针
TTree *tree=(TTree*)ipf->Get("tree");

//将tree中Branch与变量地址联系
tree->SetBranchAddress("tu",&tu);   
tree->SetBranchAddress("td",&td);
//tree->SetBranchAddress("ctof",&ctof);
//tree->SetBranchAddress("tof",&tof);
tree->SetBranchAddress("x",&x);
//tree->SetBranchAddress("pid",&pid);

## 遍历root文件事件，抽取$t_u-t_d$的tdiff；

In [2]:

Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    tdiff->Fill(td-tu);  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
tdiff->Draw();
c1->Draw();

## 观察$t_d-t_u$与$x$的关联

In [3]:
gStyle->SetPalette(1);
tree->Draw("td-tu:x","","colz");//观察两个参量的关联
//c1->SetLogz();
c1->Draw();

## 微分法抽取边界信息
### 左边界

In [4]:
//%jsroot on
TH1D *dtd=new TH1D("dtd","dt/dx",141,-20.25,50.25);//创建tdiff的微分histogram，dtd
for(int i=1;i<tdiff->GetNbinsX();i++) {//tdiff->GetNbinsX（），得到X轴bin总数，由1开始哦
    Double_t df=tdiff->GetBinContent(i+1)-tdiff->GetBinContent(i);//tdiff->GetBinContent(i+1)得到某个bin对应的constant;
    dtd->Fill(tdiff->GetBinLowEdge(i+1),df);//填图，dtd可看成tdiff的斜率图
}
dtd->Sumw2(0);//不显示传递误差
dtd->Draw();
dtd->Fit("gaus","","",-14,-8);//高斯拟合histogram
c1->Draw();

 FCN=234.719 FROM MIGRAD    STATUS=CONVERGED      82 CALLS          83 TOTAL
                     EDM=1.39314e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.83795e+02   1.16715e+01   7.13312e-02   2.02029e-05
   2  Mean        -1.18096e+01   2.22939e-02   1.65661e-04  -3.93640e-02
   3  Sigma        6.32455e-01   1.64195e-02   4.46360e-05  -1.57651e-01


ps：由于模拟数据的随机性，本人模拟产生的tdiff图的微分边界并非高斯形状，高斯拟合值偏小；
### 右边界

In [5]:
TF1 *f1 = new TF1("f1","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",39,43);//定义函数的方法 TF1
//设置初始参数
//进行参数拟合时，设置合理的初始参数至关重要！
f1->SetParameter(0,-350);//constants
f1->SetParameter(1,41.5);//mean
f1->SetParameter(2,0.5);//sigma
dtd->Fit("f1","R");
dtd->Draw();
c1->SetLogy(0);
c1->Draw();

 FCN=117.306 FROM MIGRAD    STATUS=CONVERGED      76 CALLS          77 TOTAL
                     EDM=8.17122e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -3.03431e+02   1.36086e+01   5.47072e-02   3.41968e-06
   2  p1           4.16424e+01   1.97070e-02   1.04002e-04  -4.86542e-03
   3  p2           5.71472e-01   1.70749e-02   6.83739e-05   3.38011e-03


## 探测器时间差方法刻度位置
 - $t_x = t_d - t_u$
 - $t_{xl} = -11.81$, $t_{xr} = 41.64$ //左、右边界
 - $t_{xoff} = (t_{xl} + t_{xr})/2 = 14.915$  //中心位置，offset
 $$x = \frac{2L}{t_{xr} - t_{xl}}*(t_x - t_{xoff})=3.742*(t_x - 14.915)$$

## 得到时间差方法刻度后位置图

In [6]:
TH1D *htx=new TH1D("htx","htx",500,-120,120);
TH2D *hdx=new TH2D("hdx","htx-hx:hx",100,-20,20,500,-120,120);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.742*(td-tu-14.915);
    htx->Fill(tx);//刻度后的tx
    hdx->Fill(tx-x,x);//二维图，check刻度效果
  }
htx->Draw();
c1->Draw();

## 检查刻度效果
### tx-x vs x

In [7]:
hdx->Draw("colz");//为一条竖线，没有关联
c1->Draw();

### tx-x projectionX

In [8]:
TH1D *hdx1=hdx->ProjectionX("projx of hdx");
hdx1->Draw();
hdx1->Fit("gaus");//如果中心值 mean x偏离0, 说明确定td-tu的边界有问题
c1->Draw();

 FCN=29.6002 FROM MIGRAD    STATUS=CONVERGED      51 CALLS          52 TOTAL
                     EDM=7.50242e-11    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.05708e+03   2.73290e+01   6.02862e-02  -2.58496e-08
   2  Mean        -6.37052e-02   7.15069e-03   1.93143e-05   1.70493e-03
   3  Sigma        2.26060e+00   5.05301e-03   1.64290e-06  -1.93261e-03


### 小结
- 由于模拟结果的随机性，使得微分法得到的两边界有可能偏离高斯。高斯拟合峰位偏左或便右，造成误差；
- offset设置为14.90时，拟合偏差越小；

In [9]:
//ipt->Close();//
ipf->Close();//关闭root文件

In [11]:
!jupyter nbconvert homework1.1-1 --to html

[NbConvertApp] Converting notebook homework1.1-1.ipynb to html


[NbConvertApp] Writing 475261 bytes to homework1.1-1.html


